<a href="https://colab.research.google.com/github/micah-roberson/2023_Quantinuum_Carls/blob/main/qubit_reuse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How does qubit reusability work?

*Qubit Reuse* is the process of compiling a given quantum circuit into a circuit with fewer qubits. Consider a circuit $C$. Our qubit reuse circuit is defined as $R(C)$ where $R(C) \cong C$ in the sense that $R(C)$ and $C$ are identical in the results they produce (a classical computer would not be able to identify whether a result came from $C$ or $R(C)$. 

While there is a CP-SAT method of solving this problem for the true optimal solution, we have a greedy heuristic we can employ: 

1) First, determine which qubit $q$  has the fewest number of input qubits in $C_q$. $C_q$ is defined as the causal cone of qubit $q$. This can be thought of as all the gates and qubits in the qubit $q$'s past that affect it. 

2) Next, find a qubit $q'$ in the causal cone of $C_q$ whose causal cone $C_{q'}$ adds the fewest number of input qubits to $C_q$

3) Repeat step 2 iteratively.

## Toy Example: BV Example for 4 qubits

For 4 qubits, we can run our Bernstein-Vazirani algorithm which results in the below circuit:

In [ ]:
# BV Circuit 4 Qubit Example

str_circ_bv_4 = """
OPENQASM 2.0;
include "qelib1.inc";

qreg q[4];
creg c[4];
h q[0];
h q[1];
h q[2];
x q[3];
h q[3];
cx q[0],q[3];
h q[0];
cx q[1],q[3];
h q[1];
cx q[2],q[3];
h q[2];
"""
circ_bv_4 = circuit_from_qasm_str(str_circ_bv_4)



render_circuit_jupyter(circ_bv_4)

Next, we can apply our general greedy algorithm which results in the true optimal solution for reducing our qubit count from 4 to 2. Notice that we can calculate our $q_0$ qubit and then immediately measure and reset it, thus making it available for another qubit. Notice that intuitively, if we repeat this process, we will always have two qubits left at the end for our BV qubit optimization using qubit resets. Below is the implementation of the 2 qubit version of our 4 bit BV circuit from above:

In [ ]:
circ_BV_two = Circuit(4, 3)
circ_BV_two.H(0).X(3).H(3)
# Reuse q[0] once
circ_BV_two.CX(0,3).H(0).Measure(0, 0) # q[0] --> c[0]
circ_BV_two.add_gate(OpType.Reset, [0]) # first reset

# q[0] after first reset becomes q[1] --> c[1]
circ_BV_two.H(0).CX(0,3).H(0).Measure(0, 1)
circ_BV_two.add_gate(OpType.Reset, [0]) #second reset

# q[0] after second reset becomes q[2] --> c[2]
circ_BV_two.H(0).CX(0,3).H(0).Measure(0, 2)
render_circuit_jupyter(circ_BV_two)

## Future Work

There were a few problems we wanted to solve but weren't able to due to time constraintts. Nonetheless, we will describe what our future work could be that builds on top of what we have done so far. 

### Qubit Reusibility 
Qubit reusibility allows NISQ era quantum computers to reuse qubits in order to solve problems that would require a large number of qubits that can't easily be used currently. Quantinuum has utilized this to reduce an 80 qubit version of MaxCut to an approximate 20 qubit version which had around a 91% accuracy to the true optimal solution for this NP-Hard problem. We also attempted to recreate the CP-SAT optimizer from the same paper but had difficulty writing the constraints for our cost function. Future work can be done in creating an actual circuit to CP-SAT optimizer to qubit reuse circuit pipeline. 